In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel("../데이터자료/join_data.xlsx")
df.columns
df = df[["DateTime", "종가_ex",'1Y_베이시스','국고1년','통안364일' ,'Mid_ndf','M1_스왑포인트']]
df["M1_스왑포인트"] = df["M1_스왑포인트"]/100
df.head()
df = df.set_index("DateTime")

In [2]:
df["1Y_베이시스대비"] =df["1Y_베이시스"]-df["1Y_베이시스"].shift(1)
df["국고1년대비"] =df["국고1년"]-df["국고1년"].shift(1)
df["통안364일대비"] = df["통안364일"] - df["통안364일"].shift(1)
df["Mid_ndf대비"] = df["Mid_ndf"] - df["Mid_ndf"].shift(1)
df["M1_스왑포인트대비"] = df["M1_스왑포인트"]- df["M1_스왑포인트"].shift(1)
df.head()

,종가_ex,1Y_베이시스,국고1년,통안364일,Mid_ndf,M1_스왑포인트,1Y_베이시스대비,국고1년대비,통안364일대비,Mid_ndf대비,M1_스왑포인트대비
DateTime,,,,,,,,,,,
2012-08-01,1126.5,-76.0,2.82,2.80,1132.25,2.6,NaN,NaN,NaN,NaN,NaN
2012-08-02,1131.7,-74.0,2.79,2.77,1134.00,2.5,2.0,-0.03,-0.03,1.75,-0.1
2012-08-03,1134.8,-72.0,2.77,2.75,1138.00,2.4,2.0,-0.02,-0.02,4.00,-0.1
2012-08-06,1129.0,-74.0,2.78,2.76,1128.50,2.4,-2.0,0.01,0.01,-9.50,0.0
2012-08-07,1128.8,-73.0,2.77,2.75,1129.00,2.5,1.0,-0.01,-0.01,0.50,0.1


In [3]:
# 종속변수 독립변수 설정
df["전날 종가_ex"] = df["종가_ex"].shift(1)
df = df.dropna()
df.columns
df_Target = df["종가_ex"]
df_Data = df[['1Y_베이시스대비', '국고1년대비', '통안364일대비', 'Mid_ndf', 'M1_스왑포인트']]
df_Data.feature= df_Data.columns
df_Data.feature

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23896\1047561134.py:7: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_Data.feature= df_Data.columns


Index(['1Y_베이시스대비', '국고1년대비', '통안364일대비', 'Mid_ndf', 'M1_스왑포인트'], dtype='object')

In [4]:
import statsmodels.api as sm
feature_add = sm.add_constant(df_Data, has_constant='add')

# sm OLS 적합
model = sm.OLS(df_Target, feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.933
Method:                 Least Squares   F-statistic:                     6814.
Date:                Sun, 21 Aug 2022   Prob (F-statistic):               0.00
Time:                        13:21:44   Log-Likelihood:                -10052.
No. Observations:                2459   AIC:                         2.012e+04
Df Residuals:                    2453   BIC:                         2.015e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         87.0484      6.245     13.938      0.000      74.802      99.295
1Y_베이시스대비     -0.1372      0.051     -2.680      0.007      -0.238      -0.037
국고1년대비         6.6678     16.774      0.398      0.691     -26.224      39.560
통안364일대비      -0.0021      3.190     -0.001      0.999      -6.258       6.254
Mid_ndf        0.9244      0.005    169.184      0.000       0.914       0.935
M1_스왑포인트      -3.0714      0.280    -10.965      0.000      -3.621      -2.522
==============================================================================
Omnibus:                     6154.102   Durbin-Watson:                   1.889
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         73891326.935
Skew:                          26.637   Prob(JB):                         0.00
Kurtosis:                     850.553   Cond. No.                     6.55e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.55e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
X_train = df_Data
def feature_engineering_XbyVIF(X_train):
    vif = pd.DataFrame()
    vif["VIF_Factor"] = [variance_inflation_factor(X_train.values,i)
                         for i in range(X_train.shape[1])]
    vif["Feature"] = X_train.columns
    return vif
vif = feature_engineering_XbyVIF(X_train)
print(vif)

   VIF_Factor    Feature
0    1.001238  1Y_베이시스대비
1    1.001097  Mid_ndf대비
2    1.000150   M1_스왑포인트


In [5]:
df_Target = df["종가_ex"]
df_Data = df[['1Y_베이시스대비', 'Mid_ndf대비', 'M1_스왑포인트']]

In [11]:
import statsmodels.api as sm
feature_add = sm.add_constant(df_Data, has_constant='add')

# sm OLS 적합
model = sm.OLS(df_Target, feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 8.255e+04
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        20:39:17   Log-Likelihood:                -7684.0
No. Observations:                2458   AIC:                         1.538e+04
Df Residuals:                    2454   BIC:                         1.540e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.5437      2.277      1.557      0.120      -0.920       8.008
1Y_베이시스대비     -0.2298      0.019    -11.795      0.000      -0.268      -0.192
Mid_ndf대비      0.0275      0.005      5.259      0.000       0.017       0.038
전날 종가_ex       0.9969      0.002    497.563      0.000       0.993       1.001
==============================================================================
Omnibus:                       55.041   Durbin-Watson:                   2.143
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              120.772
Skew:                          -0.003   Prob(JB):                     5.95e-27
Kurtosis:                       4.086   Cond. No.                     2.32e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.32e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
X_train, X_test, y_train, y_test = train_test_split(feature_add, df_Target,test_size=0.2, random_state=15 )

lr = LinearRegression()
lr.fit (X_train, y_train)
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)
rmse

5.277902127327842

In [14]:
from sklearn.model_selection import cross_val_score
neg_mse_scores = cross_val_score(lr, df_Data, df_Target,scoring="neg_mean_squared_error", cv = 5)
rmse_scores = np.sqrt(-1 *neg_mse_scores)
avg_rmse = np.mean(rmse_scores)
avg_rmse,rmse_scores, neg_mse_scores

(8.254598457342173,
 array([ 4.55010798,  6.75708731,  5.65035688, 19.25905693,  5.05638319]),
 array([ -20.70348262,  -45.65822891,  -31.92653292, -370.91127376,
         -25.56701092]))